# <font size="15">Shear Pocket Measurement</font>

<font size="5">Step 1: Merging</font>

In [11]:
import cv2
import numpy as np

def Merging(image1,image2):
    # Ensure both images have the same size
    if image1.shape != image2.shape:
        raise ValueError("The dimensions of the images do not match!")

    # Perform a bitwise OR operation to combine the images
    combined_image = cv2.bitwise_or(image1, image2)
    # Save the combined image
    cv2.imwrite('../Images/combined_image22.png', combined_image)

    # Display the images (optional)

    cv2.namedWindow('Combined Image', cv2.WINDOW_NORMAL)
    cv2.imshow('Combined Image', combined_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

<font size="4">Input result from Yolov9</font>

In [12]:
image = cv2.imread('/home/d8/yolov9/runs/predict-seg/exp101/masks/22_combined.png', cv2.IMREAD_GRAYSCALE)

<font size="4">Input result from SAM</font>

In [13]:
image1 = cv2.imread('../Images/P1h.jpg', cv2.IMREAD_GRAYSCALE)

In [14]:
Merging(image,image1)

<font size="5">Step2: Rotating and Measuring</font>

In [15]:
import cv2
import numpy as np
import random
from collections import defaultdict

def process_specific_contour(image, image1, contour_index):
    # Calculate new dimensions for rotated image
    def calculate_new_dimensions(w, h, angle):
        cos = np.abs(np.cos(np.radians(angle)))
        sin = np.abs(np.sin(np.radians(angle)))
        nW = int((h * sin) + (w * cos))
        nH = int((h * cos) + (w * sin))
        return nW, nH

    # Rotate image by a given angle
    def rotate_image(image, angle, nW, nH):
        (h, w) = image.shape[:2]
        (cX, cY) = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
        M[0, 2] += (nW / 2) - cX
        M[1, 2] += (nH / 2) - cY
        return cv2.warpAffine(image, M, (nW, nH))

    # Convert the image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for the color range you want to segment
    l_b = np.array([0, 0, 138])  # Lower bound
    u_b = np.array([255, 255, 255])  # Upper bound

    # Create a mask using the defined lower and upper bounds
    mask = cv2.inRange(hsv, l_b, u_b)
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:3]

    rect1 = cv2.minAreaRect(contours[contour_index])
    angle = rect1[-1]
    nW1, nH1 = calculate_new_dimensions(image.shape[1], image.shape[0], angle)
    nW2, nH2 = calculate_new_dimensions(image1.shape[1], image1.shape[0], angle)

    img = rotate_image(image, angle, nW1, nH1)
    img1 = rotate_image(image1, angle, nW2, nH2)

    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, l_b, u_b)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:3]
    

    if contours and contour_index < len(contours):
        contour = contours[contour_index]

        # Get adjacent white pixels
        def get_adjacent_white_pixels(object_mask):
            adjacent_white_pixels = []
            for y in range(1, object_mask.shape[0] - 1):
                for x in range(1, object_mask.shape[1] - 1):
                    if object_mask[y, x] == 255:
                        if (object_mask[y - 1, x] == 0 or object_mask[y + 1, x] == 0 or
                                object_mask[y, x - 1] == 0 or object_mask[y, x + 1] == 0):
                            adjacent_white_pixels.append((x, y))
            return adjacent_white_pixels

        # Draw lines between randomly selected points in the groups
        def draw_lines_between_groups(img, group1, group2, same_coordinate='x', color=(0, 255, 255)):
            if group1 and group2:
                point1 = random.choice(group1)
                point2 = random.choice(group2)
                if same_coordinate == 'x':
                    point2 = (point1[0], point2[1])
                else:
                    point2 = (point2[0], point1[1])
                cv2.line(img, point1, point2, color, 2)
                length = np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)
                return point1, point2, length
            return None, None, None

        object_mask = np.zeros_like(mask)
        cv2.drawContours(object_mask, [contour], -1, 255, thickness=cv2.FILLED)

        adjacent_white_pixels = get_adjacent_white_pixels(object_mask)

        adjacent_white_pixels_sorted = sorted(adjacent_white_pixels, key=lambda pixel: pixel[1], reverse=True)
        adjacent_white_pixels_sorted_smallest = sorted(adjacent_white_pixels, key=lambda pixel: pixel[1])

        adjacent_white_pixels_sorted_x = sorted(adjacent_white_pixels, key=lambda pixel: pixel[0])
        adjacent_white_pixels_sorted_x_smallest = sorted(adjacent_white_pixels, key=lambda pixel: pixel[0], reverse=True)

        selected_pixels = adjacent_white_pixels_sorted[:300]
        selected_pixels1 = adjacent_white_pixels_sorted_smallest[:300]
        selected_pixels2 = adjacent_white_pixels_sorted_x[:150]
        selected_pixels3 = adjacent_white_pixels_sorted_x_smallest[:150]

        y_counts = defaultdict(list)
        for (x, y) in selected_pixels:
            y_counts[y].append((x, y))

        y_counts1 = defaultdict(list)
        for (x, y) in selected_pixels1:
            y_counts1[y].append((x, y))

        x_counts = defaultdict(list)
        for (x, y) in selected_pixels2:
            x_counts[x].append((x, y))

        x_counts1 = defaultdict(list)
        for (x, y) in selected_pixels3:
            x_counts1[x].append((x, y))

        most_common_y = max(y_counts, key=lambda y: len(y_counts[y]))
        most_common_group = y_counts[most_common_y]

        most_common_y1 = max(y_counts1, key=lambda y: len(y_counts1[y]))
        most_common_group1 = y_counts1[most_common_y1]

        most_common_x = max(list(x_counts.keys()), key=lambda x: len(x_counts[x]))
        most_common_group2 = x_counts[most_common_x]

        most_common_x1 = max(list(x_counts1.keys()), key=lambda x: len(x_counts1[x]))
        most_common_group3 = x_counts1[most_common_x1]

        point1, point2, length = draw_lines_between_groups(img, most_common_group, most_common_group1, same_coordinate='x', color=(0, 0, 255))
        if point1 and point2:
            print(f"Length of the line from {point1} to {point2}: {length:.2f} pixels")

        point1, point2, length = draw_lines_between_groups(img, most_common_group2, most_common_group3, same_coordinate='y', color=(0, 255, 0))
        if point1 and point2:
            print(f"Length of the line from {point1} to {point2}: {length:.2f} pixels")

        draw_lines_between_groups(img1, most_common_group, most_common_group1, same_coordinate='x', color=(0, 0, 255))
        draw_lines_between_groups(img1, most_common_group2, most_common_group3, same_coordinate='y', color=(0, 255, 0))

        for (x, y) in selected_pixels:
            img[y, x] = [255, 0, 255]

        for (x, y) in selected_pixels1:
            img[y, x] = [255, 0, 255]

    cv2.namedWindow("Marked Image", cv2.WINDOW_NORMAL)
    cv2.imshow("Marked Image", img)
    cv2.namedWindow("Marked Image1", cv2.WINDOW_NORMAL)
    cv2.imshow("Marked Image1", img1)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [16]:
image = cv2.imread("../Images/combined_image22.png")
image1 = cv2.imread("../Images/22.jpg")

In [17]:
process_specific_contour(image, image1, contour_index=0)

Length of the line from (756, 1045) to (756, 730): 315.00 pixels
Length of the line from (580, 833) to (762, 833): 182.00 pixels


In [18]:
process_specific_contour(image, image1, contour_index=1)

Length of the line from (1290, 1006) to (1290, 690): 316.00 pixels
Length of the line from (1199, 820) to (1363, 820): 164.00 pixels


In [19]:
process_specific_contour(image, image1, contour_index=2)

Length of the line from (1843, 986) to (1843, 685): 301.00 pixels
Length of the line from (1833, 966) to (2007, 966): 174.00 pixels
